<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/pennylane101_400_FiniteDifferenceGradient_template/pennylane101_400_FiniteDifferenceGradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane
/home/javier/PycharmProjects/QHack2022/Coding_Challenges/pennylane101_400_FiniteDifferenceGradient_template/pennylane101_400_FiniteDifferenceGradient.ipynb

In [ ]:
import pennylane as qml
from pennylane import numpy as np

The goal of this challenge is to calculate the gradient with respect to the parameters of a [VQE](https://pennylane.ai/qml/demos/tutorial_vqe.html). These parameters will be given as the input $x$ of a circuit $f$ and the gradient should be approximated as:
$$
\frac{\partial f(x)}{\partial x_i} \approx \frac{f(x_i + \delta/2) - f(x_i - \delta/2)}{\delta}
$$
For this, we only need to complete the `my_finite_diff_grad()` function

In [ ]:
dev = qml.device("default.qubit", wires=3)

def variational_circuit(params):
    """A layered variational circuit. The first layer comprises of x, y, and z rotations on wires
    0, 1, and 2, respectively. The second layer is a ring of CNOT gates. The final layer comprises 
    of x, y, and z rotations on wires 0, 1, and 2, respectively.
    """

    # DO NOT MODIFY anything in this code block
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RZ(params[2], wires=2)

    qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring")

    qml.RX(params[3], wires=0)
    qml.RY(params[4], wires=1)
    qml.RZ(params[5], wires=2)

    qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring")


@qml.qnode(dev)
def cost(params):
    """A QNode that pairs the variational_circuit with an expectation value measurement.

    Args:
        - params (np.ndarray): Variational circuit parameters

    Returns:
        - (float): qml.expval(qml.PauliY(0) @ qml.PauliZ(2))
    """

    # DO NOT MODIFY anything in this code block
    variational_circuit(params)
    return qml.expval(qml.PauliY(0) @ qml.PauliZ(2))

In [ ]:
def my_finite_diff_grad(params):
    """Function that returns the gradients of the cost function (defined below) with respect 
    to all parameters in params.

    Args:
        - params (np.ndarray): The parameters needed to create the variational circuit.

    Returns:
        - gradients (np.ndarray): the gradient w.r.t. each parameter
    """

    gradients = np.zeros([len(params)])
    for i in range(len(params)):
        # QHACK #
        d = 0.001
        shifted = params.copy()
        shifted[i] += d/2
        out_plus = cost(shifted)
        shifted[i] -= d
        out_minus = cost(shifted)
        gradients[i] = (out_plus - out_minus)/d
        # QHACK #

    return gradients

Testing for random input parameters

In [ ]:
params = np.random.rand(6)*np.pi*2 - np.pi
output = my_finite_diff_grad(params)
print("Approximated gradient")
print(*output, sep=",")
print("Real gradient")
grad = qml.gradients.param_shift(cost)(params)[0]
print(*grad, sep=",")